该notebook中的模型，可以做到与biogeme的结果基本一致，包括对于模型的Structural equation，Measurement equation以及Utility function中的参数的估计，对应研究进展20181023。

# Data and Package Prepare

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

In [2]:
train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
# S=tf.Session()

In [3]:
trainable = True

In [4]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['MarginalCostPT_scaled', 'PurpHWH', 'PurpOther']]
train_CARcom = train_data[['CostCarCHF_scaled', 'PurpHWH', 'PurpOther']]

train_PTasc = train_data[['TimePT_scaled', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar_scaled']]
train_SMasc = train_data[['distance_km_scaled']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

# Latent Variable Submodel

In [5]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [6]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [7]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [8]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [9]:
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(input_latent)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [15]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=train_latent.shape[0])

1505/1505 [==============================] - 0s 0us/step


16991.0078125

In [11]:
idmodel.layers[1].get_weights()

[array([[-0.0467177 ],
        [-0.33139917],
        [-0.04428267],
        [-0.63222706],
        [ 0.514555  ],
        [ 0.5304302 ],
        [-0.6037514 ],
        [-0.37954205],
        [ 0.6305442 ],
        [-0.5306615 ],
        [-0.05998546],
        [ 0.11799723],
        [ 0.39196944]], dtype=float32), array([0.], dtype=float32)]

In [12]:
idmodel.layers[3].get_weights()

[array([[-0.625476  , -0.38966632, -0.59996676, -0.32665217, -0.464177  ,
         -0.9513744 ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [13]:
idmodel.layers[7].get_weights()

[array([1., 1., 1., 1., 1., 1., 1.], dtype=float32)]

In [14]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=8000, batch_size = train_latent.shape[0])

Epoch 1/8000
1505/1505 [==============================] - 0s 280us/step - loss: 65710.9609
Epoch 2/8000
1505/1505 [==============================] - 0s 10us/step - loss: 64020.6992
Epoch 3/8000
1505/1505 [==============================] - 0s 0us/step - loss: 62176.4414
Epoch 4/8000
1505/1505 [==============================] - 0s 10us/step - loss: 60344.5859
Epoch 5/8000
1505/1505 [==============================] - 0s 10us/step - loss: 58578.4375
Epoch 6/8000
1505/1505 [==============================] - 0s 21us/step - loss: 56895.2812
Epoch 7/8000
1505/1505 [==============================] - 0s 21us/step - loss: 55302.5547
Epoch 8/8000
1505/1505 [==============================] - 0s 10us/step - loss: 53803.4414
Epoch 9/8000
1505/1505 [==============================] - 0s 10us/step - loss: 52397.4258
Epoch 10/8000
1505/1505 [==============================] - 0s 10us/step - loss: 51081.4219
Epoch 11/8000
1505/1505 [==============================] - 0s 10us/step - loss: 49850.6953
Epoch 12

Epoch 92/8000
1505/1505 [==============================] - 0s 5us/step - loss: 24628.7539
Epoch 93/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24558.3926
Epoch 94/8000
1505/1505 [==============================] - 0s 0us/step - loss: 24489.4336
Epoch 95/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24421.8398
Epoch 96/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24355.5742
Epoch 97/8000
1505/1505 [==============================] - 0s 21us/step - loss: 24290.5938
Epoch 98/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24226.8613
Epoch 99/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24164.3457
Epoch 100/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24103.0137
Epoch 101/8000
1505/1505 [==============================] - 0s 10us/step - loss: 24042.8340
Epoch 102/8000
1505/1505 [==============================] - 0s 10us/step - loss: 23983.785

Epoch 182/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21330.2168
Epoch 183/8000
1505/1505 [==============================] - 0s 0us/step - loss: 21311.2793
Epoch 184/8000
1505/1505 [==============================] - 0s 0us/step - loss: 21292.5391
Epoch 185/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21273.9902
Epoch 186/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21255.6309
Epoch 187/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21237.4570
Epoch 188/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21219.4688
Epoch 189/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21201.6641
Epoch 190/8000
1505/1505 [==============================] - 0s 21us/step - loss: 21184.0352
Epoch 191/8000
1505/1505 [==============================] - 0s 10us/step - loss: 21166.5840
Epoch 192/8000
1505/1505 [==============================] - 0s 21us/step - loss: 2

Epoch 272/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20176.5117
Epoch 273/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20168.0918
Epoch 274/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20159.7363
Epoch 275/8000
1505/1505 [==============================] - 0s 0us/step - loss: 20151.4492
Epoch 276/8000
1505/1505 [==============================] - 0s 0us/step - loss: 20143.2266
Epoch 277/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20135.0723
Epoch 278/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20126.9785
Epoch 279/8000
1505/1505 [==============================] - 0s 21us/step - loss: 20118.9473
Epoch 280/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20110.9824
Epoch 281/8000
1505/1505 [==============================] - 0s 10us/step - loss: 20103.0781
Epoch 282/8000
1505/1505 [==============================] - 0s 10us/step - loss: 2

1505/1505 [==============================] - 0s 10us/step - loss: 19623.9023
Epoch 362/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19619.3633
Epoch 363/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19614.8516
Epoch 364/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19610.3633
Epoch 365/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19605.9043
Epoch 366/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19601.4707
Epoch 367/8000
1505/1505 [==============================] - 0s 21us/step - loss: 19597.0625
Epoch 368/8000
1505/1505 [==============================] - 0s 21us/step - loss: 19592.6836
Epoch 369/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19588.3262
Epoch 370/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19583.9961
Epoch 371/8000
1505/1505 [==============================] - 0s 10us/step - loss: 19579.6914
Epo

...

Epoch 7569/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7570/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7571/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7572/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7573/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7574/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7575/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9961
Epoch 7576/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7577/8000
1505/1505 [==============================] - 0s 21us/step - loss: 16990.9961
Epoch 7578/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7579/8000
1505/1505 [==============================] - 0s 10us/step

Epoch 7658/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7659/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7660/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7661/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7662/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7663/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9980
Epoch 7664/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0000
Epoch 7665/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7666/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7667/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7668/8000
1505/1505 [==============================] - 0s 10us/step -

Epoch 7747/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7748/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0039
Epoch 7749/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16991.0078
Epoch 7750/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0078
Epoch 7751/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0000
Epoch 7752/8000
1505/1505 [==============================] - 0s 21us/step - loss: 16990.9961
Epoch 7753/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0000
Epoch 7754/8000
1505/1505 [==============================] - 0s 21us/step - loss: 16991.0039
Epoch 7755/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16991.0039
Epoch 7756/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16991.0000
Epoch 7757/8000
1505/1505 [==============================] - 0s 10us/ste

Epoch 7836/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7837/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7838/8000
1505/1505 [==============================] - 0s 21us/step - loss: 16990.9980
Epoch 7839/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7840/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7841/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7842/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7843/8000
1505/1505 [==============================] - 0s 13us/step - loss: 16990.9980
Epoch 7844/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7845/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9980
Epoch 7846/8000
1505/1505 [==============================] - 0s 21us/s

1505/1505 [==============================] - 0s 21us/step - loss: 16990.9980
Epoch 7925/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7926/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9961
Epoch 7927/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9961
Epoch 7928/8000
1505/1505 [==============================] - 0s 0us/step - loss: 16990.9961
Epoch 7929/8000
1505/1505 [==============================] - 0s 21us/step - loss: 16990.9961
Epoch 7930/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7931/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7932/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7933/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9961
Epoch 7934/8000
1505/1505 [==============================] - 0s 10us/step - loss: 16990.9

In [16]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
indicator_ref 2
indicator_other 3
concatenate 4
input_y_ture 5
subtract 6
ll 7


In [17]:
idmodel.layers[1].get_weights()

[array([[-0.1259092 ],
        [-0.09641296],
        [ 0.3678927 ],
        [-0.34928215],
        [ 0.625567  ],
        [-0.06615467],
        [ 0.05817849],
        [ 0.6222404 ],
        [ 0.21793641],
        [ 0.0993166 ],
        [-0.04281182],
        [ 0.279273  ],
        [-0.61859584]], dtype=float32), array([2.5601623], dtype=float32)]

In [18]:
idmodel.layers[2].get_weights()

[array([[1.]], dtype=float32), array([0.], dtype=float32)]

In [19]:
idmodel.layers[3].get_weights()

[array([[ 0.46806106, -0.77926636, -0.7596789 , -0.70193034, -0.65280664,
         -0.7071629 ]], dtype=float32),
 array([2.0837853, 5.022559 , 5.7847285, 5.0839744, 5.1904716, 5.5095224],
       dtype=float32)]

In [20]:
idmodel.layers[7].get_weights()

[array([1.3653334, 1.1685717, 1.2034935, 1.1288774, 1.1913147, 1.14992  ,
        1.3085434], dtype=float32)]

# Discrete Choice Submodel

计算效用中公共系数部分

In [21]:
class util_common(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(util_common, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_cost_hwh = self.add_weight(name='beta_cost_hwh',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.13),
                                      # constraints = min_max_norm(min_value=-1.650001, max_value=-1.649999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cost_other = self.add_weight(name='beta_cost_other',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.465),
                                      # constraints = min_max_norm(min_value=-1.590001, max_value=-1.589999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(util_common, self).build(input_shape)

    def call(self, inputs):
        hwh = tf.multiply(tf.multiply(inputs[:,0], inputs[:,1]), self.beta_cost_hwh)
        other = tf.multiply(tf.multiply(inputs[:,0], inputs[:,2]), self.beta_cost_other)
        util=tf.transpose(tf.add(hwh, other))
        return tf.reshape(tf.tile(util,[1,random_num]),[-1,random_num])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(util_common, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


V0 alternative specific part

In [22]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.72),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.37),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.0275),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [23]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=4.66),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-5.83),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        # cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        # return tf.add(tf.add(cl,timeCAR), self.asc_CAR)
        return tf.add(timeCAR, self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [24]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.22),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.62),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        self.beta_cl_SM = self.add_weight(name='beta_cl_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=2.),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        cl_SM = tf.multiply(inputs[0], self.beta_cl_SM)
        dist = tf.multiply(inputs[1], self.beta_dist)
        
        return tf.add(tf.add(cl_SM, dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [25]:
class Add_Random(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Add_Random, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(Add_Random, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.random_normal((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(Add_Random, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [26]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
car_sys = idmodel.layers[1](input_latent)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1505,random_num)), name = 'carlover')(car_sys)
carlover = Add_Random(1,name = 'carlover')(car_sys)

### common coefficients part in utility
utilcommon = util_common(1, name = 'util_common')
PTcom = utilcommon(input_PTcom)
CARcom = utilcommon(input_CARcom)


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)([carlover, input_SMasc])
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [27]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1505/1505 [==============================] - 0s 239us/step


[0.5990431308746338, 0.7455149292945862]

In [ ]:
# tf.keras.backend.get_value(util_V1as)
# callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=4000, batch_size = train_latent.shape[0])

In [29]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1505/1505 [==============================] - 0s 21us/step


[0.5977398753166199, 0.7455149292945862]

In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [41]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
carlover 2
input_PTasc 3
input_PTcom 4
input_CARasc 5
input_CARcom 6
input_SMasc 7
v0_as_2 8
util_common 9
v1_as_2 10
v2_as_2 11
V0 12
V1 13
V2 14
prediction 15


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [55]:
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, model.layers[output_index[i]].get_weights()))

car_sys         [array([[-0.12619334],
       [-0.09635082],
       [ 0.36885747],
       [-0.3496434 ],
       [ 0.6265547 ],
       [-0.06594128],
       [ 0.05845913],
       [ 0.6226478 ],
       [ 0.21824066],
       [ 0.09964096],
       [-0.04249339],
       [ 0.2796297 ],
       [-0.6184652 ]], dtype=float32), array([2.5612953], dtype=float32)]
carlover        []
v0_as_4         [array([[-2.0230172]], dtype=float32), array([[1.5660286]], dtype=float32), array([[-0.02796068]], dtype=float32)]
util_common     [array([[-1.172912]], dtype=float32), array([[-0.47131765]], dtype=float32)]
v1_as_4         [array([[5.2523074]], dtype=float32), array([[-6.434374]], dtype=float32)]
v2_as_4         [array([[-1.5794731]], dtype=float32), array([[-0.4133285]], dtype=float32), array([[1.9687463]], dtype=float32)]
V0              []
V1              []
V2              []
prediction      []


In [47]:
model.layers[9].get_weights()

[array([[-1.392449]], dtype=float32), array([[-0.55060613]], dtype=float32)]